In [1]:
from icevision.all import *
from lunular.data import coco

INFO     - The mmdet config folder already exists. No need to downloaded it. Path : /home/rave/.icevision/mmdetection_configs/mmdetection_configs-2.16.0/configs | icevision.models.mmdet.download_configs:download_mmdet_configs:17


In [2]:
model_type = models.torchvision.mask_rcnn
backbone = model_type.backbones.resnet34_fpn()

In [3]:
ml_data_path = os.path.join(os.path.abspath(os.getcwd()),"../ML/Data")

In [4]:
import numpy as np
path = Path(ml_data_path)
oil_chps = np.loadtxt(path/'oil_chps.txt', dtype=str)
codes = np.loadtxt(path/'codes-nocoincident.txt', dtype=str)
valid_names = np.loadtxt(path/'valid_names.txt', dtype=str)

OSError: /home/rave/skytruth-internal/notebooks/../ML/Data/oil_chps.txt not found.

In [ ]:
random.seed(42)
oilless_chps = [chp for chp in get_image_files(path/'chp') if chp.stem not in oil_chps]
density = 0
num_oilless = density * len(oil_chps)
mixed_chps = random.sample(oilless_chps,num_oilless) + [path/'chp'/(chp+'.png') for chp in oil_chps]
len(mixed_chps)

In [ ]:
def get_chps(path): return mixed_chps
def get_lbls(fn): return fn.parent.parent/"lbl_no3"/f"{fn.name}"
def splitter_func(fn): return fn.name.rsplit('_',1)[0] in valid_names # XXX Check to make sure this should be returning True for Valid

drive_files = get_chps(path)

In [ ]:
lbl_chps = [get_lbls(x) for x in mixed_chps]

In [ ]:
lcl = Path("../out_data")
path.mkdir(exist_ok=True)
modelpath = lcl

### Making a COCO Dataset for Mask-RCNN

#### Annotations from raster labels
##### https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch this can be incorporated into solaris/lunular

In [9]:
from PIL import Image # (pip install Pillow)
import numpy as np
def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the VV value of the pixel
            pixel = mask_image.getpixel((x,y))

            # If the pixel is not black...
            if pixel != 0:
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

import numpy as np                                 # (pip install numpy)
from skimage import measure                        # (pip install scikit-image)
from shapely.geometry import Polygon, MultiPolygon # (pip install Shapely)

def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation

In [10]:
info = {
    "description": "CeruleanDataset V1, Binary Slick/Not Slick",
    "url": "none",
    "version": "1.0",
    "year": 2021,
    "contributor": "Skytruth",
    "date_created": "2021/11/12"
}

licenses = [
    {
        "url": "none",
        "id": 1,
        "name": "CeruleanDataset V1, Binary Slick/Not Slick"
    }
]

{
    "info": info,
    "licenses": [...],
    "images": [...],
    "annotations": [...],
    "categories": [...], <-- Not in Captions annotations
    "segment_info": [...] <-- Only in Panoptic annotations
}

SyntaxError: invalid syntax (30185141.py, line 23)

In [11]:
test_path = [i for i in lbl_chps if "S1A_IW_GRDH_1SDV_20190105T221712_20190105T221737_025348_02CE45_53F2_1" in str(i)]

In [12]:
import json
lbl_chps[0]
test_mask_image = Image.open(str(test_path[0]))

mask_images = [test_mask_image]

# Define which colors match which categories in the images
oil_slick_id = [1]
category_ids = {
    1: {
        '1': oil_slick_id,
    }
}

is_crowd = 0

# These ids will be automatically increased as we go
annotation_id = 1
image_id = 1

# Create the annotations
annotations = []
for mask_image in mask_images:
    sub_masks = create_sub_masks(mask_image)
    for color, sub_mask in sub_masks.items():
        category_id = category_ids[image_id][color]
        annotation = create_sub_mask_annotation(np.array(sub_mask), image_id, category_id, annotation_id, is_crowd)
        annotations.append(annotation)
        annotation_id += 1
    image_id += 1

print(json.dumps(annotations))

[{"segmentation": [[127.0, 377.5, 143.0, 371.5, 174.0, 367.5, 203.0, 360.5, 218.0, 360.5, 238.0, 349.5, 241.0, 353.5, 244.0, 348.5, 245.0, 349.5, 246.0, 347.5, 247.0, 349.5, 251.0, 345.5, 262.0, 345.5, 267.0, 341.5, 269.0, 342.5, 277.0, 340.5, 279.5, 339.0, 282.0, 333.5, 283.0, 334.5, 284.0, 332.5, 286.5, 332.0, 286.0, 330.5, 291.0, 332.5, 296.0, 329.5, 304.0, 329.5, 306.0, 327.5, 308.0, 328.5, 314.0, 324.5, 319.0, 323.5, 320.0, 321.5, 324.0, 321.5, 329.0, 316.5, 332.0, 317.5, 340.0, 313.5, 342.0, 315.5, 346.0, 312.5, 361.0, 310.5, 365.0, 307.5, 371.0, 307.5, 381.0, 304.5, 389.0, 305.5, 402.0, 302.5, 410.0, 303.5, 419.0, 298.5, 425.0, 298.5, 429.0, 295.5, 439.0, 295.5, 442.0, 292.5, 444.0, 294.5, 448.0, 291.5, 452.0, 293.5, 454.0, 288.5, 457.0, 290.5, 461.0, 285.5, 463.0, 288.5, 469.0, 283.5, 474.0, 283.5, 474.5, 281.0, 477.0, 279.5, 481.0, 281.5, 481.0, 279.5, 484.0, 279.5, 485.0, 277.5, 492.0, 277.5, 495.5, 274.0, 495.0, 271.5, 507.0, 271.5, 507.0, 269.5, 509.0, 270.5, 511.5, 268.0, 

https://airctic.com/0.8.1/custom_parser/

In [66]:
import icedata

icedata.coco.parser

<module 'icedata.datasets.coco.parser' from '/home/rave/miniconda3/envs/icevision/lib/python3.8/site-packages/icedata/datasets/coco/parser.py'>

Do all transforms happen on the gpu? with icevision? do albumentations transforms happen on gpu?


In [75]:
tfms.transform.Transform??

Init signature: tfms.transform.Transform()
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
Source:        
class Transform(ABC):
    def __call__(self, record: BaseRecord):
        # TODO: this assumes record is already loaded and copied
        # which is generally true
        return self.apply(record)

    @abstractmethod
    def apply(self, record: BaseRecord) -> BaseRecord:
        """Apply the transform
        Returns:
              dict: Modified values, the keys of the dictionary should have the same
              names as the keys received by this function
        """
File:           ~/miniconda3/envs/icevision/lib/python3.8/site-packages/icevision/tfms/transform.py
Type:           ABCMeta
Subclasses:     Adapter


In [74]:
size = 64 # Progressive resizing could happen here
augs = aug_transforms(flip_vert=True, max_warp=0.1, size=size)

NameError: name 'aug_transforms' is not defined

In [11]:
model_type??

Type:        module
String form: <module 'icevision.models.torchvision.mask_rcnn' from '/home/rave/miniconda3/envs/icevision/lib/python3.8/site-packages/icevision/models/torchvision/mask_rcnn/__init__.py'>
File:        ~/miniconda3/envs/icevision/lib/python3.8/site-packages/icevision/models/torchvision/mask_rcnn/__init__.py
Source:     
from icevision.models.torchvision.loss_fn import *

from icevision.models.torchvision.mask_rcnn import backbones
from icevision.models.torchvision.mask_rcnn.dataloaders import *
from icevision.models.torchvision.mask_rcnn.model import *
from icevision.models.torchvision.mask_rcnn.prediction import *
from icevision.models.torchvision.mask_rcnn.show_results import *
from icevision.models.torchvision.mask_rcnn.show_batch import *

# Soft dependencies
from icevision.soft_dependencies import SoftDependencies

if SoftDependencies.fastai:
    import icevision.models.torchvision.mask_rcnn.fastai

if SoftDependencies.pytorch_lightning:
    import icevision.model